In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
print( os.getcwd() )

/Users/austinclime/vs_code_projects/vix_approx/vix_approx_research


In [4]:
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from io import StringIO
from datetime import timedelta
import pytz
import datetime

import pandas_market_calendars as mcal

In [5]:
from vix_skew_returns import option_metric_query, big_listing_query,get_fwd_price, calculate_interest_rates

aclime2 
Loading library list...
Done


In [6]:
df = option_metric_query('SPX',2022,day=None,month=None)

In [7]:
df.columns

Index(['secid', 'date', 'symbol', 'symbol_flag', 'exdate', 'last_date',
       'cp_flag', 'strike_price', 'best_bid', 'best_offer', 'volume',
       'open_interest', 'impl_volatility', 'delta', 'gamma', 'vega', 'theta',
       'optionid', 'cfadj', 'am_settlement', 'contract_size', 'ss_flag',
       'forward_price', 'expiry_indicator', 'root', 'suffix'],
      dtype='object')

In [7]:
df2=df.copy()

In [76]:
df2[df2.exdate=='2023-12-15']

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,...,theta,optionid,cfadj,am_settlement,contract_size,ss_flag,forward_price,expiry_indicator,root,suffix
5448,108105.0,2022-01-03,SPX 231215C100000,1,2023-12-15,2021-12-30,C,100.0,4525.1,4624.7,...,<NA>,137718293.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>
5449,108105.0,2022-01-03,SPX 231215C1000000,1,2023-12-15,2021-12-28,C,1000.0,3654.6,3754.0,...,<NA>,137581087.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>
5450,108105.0,2022-01-03,SPX 231215C1100000,1,2023-12-15,2021-09-30,C,1100.0,3558.6,3658.0,...,<NA>,137581088.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>
5451,108105.0,2022-01-03,SPX 231215C1200000,1,2023-12-15,2021-10-22,C,1200.0,3463.0,3562.4,...,<NA>,137581089.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>
5452,108105.0,2022-01-03,SPX 231215C1300000,1,2023-12-15,2021-05-28,C,1300.0,3367.4,3466.8,...,<NA>,137581090.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213333,108105.0,2022-12-30,SPX 231215C3100000,1,2023-12-15,2022-12-20,C,3100.0,916.1,923.3,...,-190.7774,137581132.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>
213334,108105.0,2022-12-30,SPX 231215C3125000,1,2023-12-15,NaT,C,3125.0,896.2,903.4,...,-193.4023,137581133.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>
213335,108105.0,2022-12-30,SPX 231215C3150000,1,2023-12-15,2022-10-24,C,3150.0,876.4,883.7,...,-195.9871,137581134.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>
213336,108105.0,2022-12-30,SPX 231215C3175000,1,2023-12-15,2021-08-24,C,3175.0,857.0,865.2,...,-198.9938,137581135.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>


Holidays

In [8]:
lb='1996-01-04'
ub='2040-12-31'
#CBOE Holidays and Early Closes
#   NOTE: need to come back to early closes
cboe=mcal.get_calendar('CBOE_Index_Options')
cboe_holidays=cboe.holidays()
cboe_holidaylist=pd.to_datetime(cboe_holidays.holidays)
#sched=cboe.schedule(start_date=lb, end_date=ub)
#cboe.early_closes(schedule=sched)

#
fridays = list( pd.date_range(lb, ub,freq='W-FRI', tz='US/Eastern',normalize=True).values )
third_fridays = list( pd.date_range(lb, ub,freq='WOM-3FRI', tz='US/Eastern',normalize=True).values )
fridays=pd.to_datetime(fridays).normalize()
third_fridays=pd.to_datetime(third_fridays).normalize()
diffed_fridays=list(set(fridays)-set(third_fridays))

#print(third_fridays)
#print(cboe_holidaylist)
holiday_dict={}
third_friday_holidays=list(set(third_fridays) & set(cboe_holidaylist))
print(third_friday_holidays)
expir_not_third_friday=set(pd.to_datetime(df2.exdate.unique()).normalize())-set(third_fridays)
print(expir_not_third_friday)
print('--------------------------')
for expir in expir_not_third_friday:
    #print(expir in third_friday_holidays)
    for holiday in third_friday_holidays:
        if (holiday-expir).days==1:
            #print(holiday,expir)
            holiday_dict[expir]=True

#df3=df2.copy()
df2['holiday_exp']=df2['exdate'].map(holiday_dict)
#df2['holiday_exp'].fillna(value=False,inplace=True)
df2.fillna({'holiday_exp':False},inplace=True)

[Timestamp('2022-04-15 00:00:00'), Timestamp('2032-06-18 00:00:00'), Timestamp('2027-06-18 00:00:00'), Timestamp('2033-04-15 00:00:00'), Timestamp('2014-04-18 00:00:00'), Timestamp('2008-03-21 00:00:00'), Timestamp('2038-06-18 00:00:00'), Timestamp('2030-04-19 00:00:00'), Timestamp('2003-04-18 00:00:00'), Timestamp('2019-04-19 00:00:00'), Timestamp('2026-06-19 00:00:00'), Timestamp('2037-06-19 00:00:00'), Timestamp('2025-04-18 00:00:00'), Timestamp('2000-04-21 00:00:00')]
{Timestamp('2022-04-14 00:00:00')}
--------------------------


/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_17480/3807597390.py:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2.fillna({'holiday_exp':False},inplace=True)


In [9]:
df2

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,...,optionid,cfadj,am_settlement,contract_size,ss_flag,forward_price,expiry_indicator,root,suffix,holiday_exp
0,108105.0,2022-01-03,SPX 220121C1000000,1,2022-01-21,2021-12-22,C,1000.0,3787.3,3792.5,...,137642934.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>,False
1,108105.0,2022-01-03,SPX 220121C1100000,1,2022-01-21,2021-12-17,C,1100.0,3687.4,3692.5,...,137642935.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>,False
2,108105.0,2022-01-03,SPX 220121C1200000,1,2022-01-21,2021-12-23,C,1200.0,3587.3,3592.5,...,137642936.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>,False
3,108105.0,2022-01-03,SPX 220121C1300000,1,2022-01-21,2021-06-10,C,1300.0,3487.4,3492.6,...,137642937.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>,False
4,108105.0,2022-01-03,SPX 220121C1400000,1,2022-01-21,2021-12-17,C,1400.0,3387.4,3392.6,...,137642938.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213333,108105.0,2022-12-30,SPX 271217P8000000,1,2027-12-17,2022-11-11,P,8000.0,3001.0,3073.0,...,148122153.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>,False
213334,108105.0,2022-12-30,SPX 271217P8400000,1,2027-12-17,NaT,P,8400.0,3317.5,3389.5,...,149433122.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>,False
213335,108105.0,2022-12-30,SPX 271217P8800000,1,2027-12-17,NaT,P,8800.0,3636.0,3708.0,...,149433123.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>,False
213336,108105.0,2022-12-30,SPX 271217P9200000,1,2027-12-17,NaT,P,9200.0,3956.5,4028.5,...,149433124.0,1.0,1.0,100.0,0,<NA>,<NA>,<NA>,<NA>,False


In [9]:
#set(pd.to_datetime(df2.exdate.unique()).normalize())-set(third_fridays)
expir_not_third_friday=set(pd.to_datetime(df2.exdate.unique()).normalize())-set(third_fridays)
expir_not_third_friday

{Timestamp('2022-04-14 00:00:00')}

In [36]:
cboe=mcal.get_calendar('CBOE_Index_Options')
cboe_holidays=cboe.holidays()
cboe_holidaylist=pd.to_datetime(cboe_holidays.holidays)
'2022-04-15' in cboe_holidaylist

True

Listings

~get from Bloomberg

In [9]:
listing_df

,secid,effect_date,seq_num,status,exchange,add_del,exch_flag
0,5001.0,1996-01-01,1.0,$,F,*,4
1,5001.0,2007-03-08,1.0,D,F,<NA>,0
2,5002.0,1996-01-01,1.0,$,F,*,4
3,5002.0,1996-02-01,1.0,X,F,<NA>,0
4,5003.0,1996-01-01,1.0,$,O,*,16
...,...,...,...,...,...,...,...
95,5046.0,1996-01-01,1.0,$,A,*,1
96,5046.0,1999-06-24,1.0,X,A,<NA>,0
97,5047.0,1996-01-01,1.0,$,A,*,1
98,5047.0,1997-02-19,1.0,E,A,<NA>,0


In [39]:
from vix_skew_returns import big_listing_query

In [11]:
df_listing=big_listing_query('SPX')

In [10]:
df_listing

,optionid,approx_listing_date
0,11231305.0,1996-08-27
1,11512288.0,1996-09-30
2,10318764.0,1996-11-08
3,10899616.0,1996-11-14
4,10712188.0,1996-11-19
...,...,...
96303,156521849.0,2023-08-31
96304,156521837.0,2023-08-31
96305,155844238.0,2023-08-31
96306,156521873.0,2023-08-31


Expirations

In [10]:
df3=df2.copy()    
df3['datetime_close']=df3['date']+pd.Timedelta(hours=16,minutes=15)
df3['ex_time']=df3['exdate']+pd.Timedelta(hours=9,minutes=30)
df3['time_to_exp']=df3['ex_time']-df3['datetime_close']

In [13]:
df4 = df3.merge(df_listing,how='left',on=['optionid'])

In [18]:
df4[pd.isnull(df4.approx_listing_date)]
#99767 nullss using last date
#0 nulls using date


df4['approx_tenor']=df4['ex_time']-df4['approx_listing_date']
df4.approx_tenor.unique()

<TimedeltaArray>
['396 days 09:30:00', '266 days 09:30:00', '112 days 09:30:00',
 '122 days 09:30:00', '182 days 09:30:00',  '98 days 09:30:00',
 '388 days 09:30:00', '375 days 09:30:00', '365 days 09:30:00',
 '347 days 09:30:00',
 ...
 '358 days 09:30:00',  '84 days 09:30:00',  '21 days 09:30:00',
  '65 days 09:30:00', '100 days 09:30:00', '337 days 09:30:00',
  '88 days 09:30:00',   '7 days 09:30:00', '415 days 09:30:00',
  '15 days 09:30:00']
Length: 334, dtype: timedelta64[ns]

In [17]:
#x=df3.groupby(['exdate','time_to_exp','strike_price','cp_flag']).count()
#x=df3.groupby(['exdate','date','strike_price','cp_flag']).count()
x=df3.groupby(['time_to_exp','date','strike_price','cp_flag']).count()
display( x[x.symbol>=2] )
#df3[ (df3.exdate=='2022-01-21') & (df3.strike_price==800) & (df3.time_to_exp.dt.days==0)]

,,,,secid,symbol,symbol_flag,exdate,last_date,best_bid,best_offer,volume,open_interest,impl_volatility,...,am_settlement,contract_size,ss_flag,forward_price,expiry_indicator,root,suffix,holiday_exp,datetime_close,ex_time
time_to_exp,date,strike_price,cp_flag,,,,,,,,,,,,,,,,,,,,,


In [35]:
#sorted( df3[df3.date.isin(third_fridays)].time_to_exp.unique() )
df3[df3.date.isin(third_fridays)].groupby(['date'])['exdate'].unique()

date
2022-01-21    [2022-02-18 00:00:00, 2022-03-18 00:00:00, 202...
2022-02-18    [2022-03-18 00:00:00, 2022-04-14 00:00:00, 202...
2022-03-18    [2022-04-14 00:00:00, 2022-05-20 00:00:00, 202...
2022-05-20    [2022-06-17 00:00:00, 2022-07-15 00:00:00, 202...
2022-06-17    [2022-07-15 00:00:00, 2022-08-19 00:00:00, 202...
2022-07-15    [2022-08-19 00:00:00, 2022-09-16 00:00:00, 202...
2022-08-19    [2022-09-16 00:00:00, 2022-10-21 00:00:00, 202...
2022-09-16    [2022-10-21 00:00:00, 2022-11-18 00:00:00, 202...
2022-10-21    [2022-11-18 00:00:00, 2022-12-16 00:00:00, 202...
2022-11-18    [2022-12-16 00:00:00, 2023-01-20 00:00:00, 202...
2022-12-16    [2023-01-20 00:00:00, 2023-02-17 00:00:00, 202...
Name: exdate, dtype: object

fwd price

In [12]:
df_fwd=get_fwd_price('SPX',2022)

In [25]:
#x=df_fwd.groupby(['date','expiration']).nunique()
#x[x>=2].dropna()

#for fri in third_fridays:
    #print(fri)
    #df_fwd.expiration.unique()

print( df_fwd.expiration.unique() )
print(third_fridays[third_fridays.year ==2023] )

df_fwd

<DatetimeArray>
['2022-01-21 00:00:00', '2022-02-18 00:00:00', '2022-03-18 00:00:00',
 '2022-04-14 00:00:00', '2022-05-20 00:00:00', '2022-06-17 00:00:00',
 '2022-07-15 00:00:00', '2022-08-19 00:00:00', '2022-09-16 00:00:00',
 '2022-10-21 00:00:00', '2022-11-18 00:00:00', '2022-12-16 00:00:00',
 '2023-01-20 00:00:00', '2023-03-17 00:00:00', '2023-06-16 00:00:00',
 '2023-12-15 00:00:00', '2024-12-20 00:00:00', '2025-12-19 00:00:00',
 '2026-12-18 00:00:00', '2023-02-17 00:00:00', '2023-04-21 00:00:00',
 '2023-05-19 00:00:00', '2023-09-15 00:00:00', '2022-06-14 00:00:00',
 '2022-06-15 00:00:00', '2022-06-16 00:00:00', '2022-06-21 00:00:00',
 '2022-06-22 00:00:00', '2022-06-23 00:00:00', '2022-06-24 00:00:00',
 '2022-06-27 00:00:00', '2022-06-28 00:00:00', '2022-06-29 00:00:00',
 '2022-06-30 00:00:00', '2022-07-01 00:00:00', '2022-07-05 00:00:00',
 '2022-07-06 00:00:00', '2022-07-08 00:00:00', '2022-07-11 00:00:00',
 '2022-07-13 00:00:00', '2022-07-22 00:00:00', '2022-07-29 00:00:00',
 '20

,secid,date,expiration,amsettlement,forwardprice
0,108105.0,2022-01-03,2022-01-21,1.0,4790.779509
1,108105.0,2022-01-03,2022-02-18,1.0,4787.149658
2,108105.0,2022-01-03,2022-03-18,1.0,4782.992117
3,108105.0,2022-01-03,2022-04-14,1.0,4781.039728
4,108105.0,2022-01-03,2022-05-20,1.0,4777.861522
...,...,...,...,...,...
4858,108105.0,2022-12-30,2024-06-21,1.0,4027.923332
4859,108105.0,2022-12-30,2024-12-20,1.0,4074.566259
4860,108105.0,2022-12-30,2025-12-19,1.0,4160.401119
4861,108105.0,2022-12-30,2026-12-18,1.0,4242.230857


In [13]:
df4=df3.merge(df_fwd,how='left',left_on=['secid','date','exdate'],right_on=['secid','date','expiration'])
#df4[pd.isnull(df4.forwardprice)]
#df4['OOM']=df4.apply(lambda x:True if ((x.cp_flag=='C' and x.strike_price>=x.forwardprice) or (x.cp_flag=='P' and x.strike_price<=x.forwardprice)) else False )
def oom_indicator(row):
    if (row.cp_flag=='C' and row.strike_price>=row.forwardprice) or (row.cp_flag=='P' and row.strike_price<=row.forwardprice):
        return True
    else:
        return False
#df4['oom']=df4.apply(oom_indicator,axis=1)
df4['midpoint_price']=(df4.best_bid+df4.best_offer)/2
df4

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,...,root,suffix,holiday_exp,datetime_close,ex_time,time_to_exp,expiration,amsettlement,forwardprice,midpoint_price
0,108105.0,2022-01-03,SPX 220121C1000000,1,2022-01-21,2021-12-22,C,1000.0,3787.3,3792.5,...,<NA>,<NA>,False,2022-01-03 16:15:00,2022-01-21 09:30:00,17 days 17:15:00,2022-01-21,1.0,4790.779509,3789.9
1,108105.0,2022-01-03,SPX 220121C1100000,1,2022-01-21,2021-12-17,C,1100.0,3687.4,3692.5,...,<NA>,<NA>,False,2022-01-03 16:15:00,2022-01-21 09:30:00,17 days 17:15:00,2022-01-21,1.0,4790.779509,3689.95
2,108105.0,2022-01-03,SPX 220121C1200000,1,2022-01-21,2021-12-23,C,1200.0,3587.3,3592.5,...,<NA>,<NA>,False,2022-01-03 16:15:00,2022-01-21 09:30:00,17 days 17:15:00,2022-01-21,1.0,4790.779509,3589.9
3,108105.0,2022-01-03,SPX 220121C1300000,1,2022-01-21,2021-06-10,C,1300.0,3487.4,3492.6,...,<NA>,<NA>,False,2022-01-03 16:15:00,2022-01-21 09:30:00,17 days 17:15:00,2022-01-21,1.0,4790.779509,3490.0
4,108105.0,2022-01-03,SPX 220121C1400000,1,2022-01-21,2021-12-17,C,1400.0,3387.4,3392.6,...,<NA>,<NA>,False,2022-01-03 16:15:00,2022-01-21 09:30:00,17 days 17:15:00,2022-01-21,1.0,4790.779509,3390.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713333,108105.0,2022-12-30,SPX 230421C3650000,1,2023-04-21,2022-12-29,C,3650.0,327.6,329.0,...,<NA>,<NA>,False,2022-12-30 16:15:00,2023-04-21 09:30:00,111 days 17:15:00,2023-04-21,1.0,3874.60935,328.3
1713334,108105.0,2022-12-30,SPX 230421C3655000,1,2023-04-21,NaT,C,3655.0,324.1,325.5,...,<NA>,<NA>,False,2022-12-30 16:15:00,2023-04-21 09:30:00,111 days 17:15:00,2023-04-21,1.0,3874.60935,324.8
1713335,108105.0,2022-12-30,SPX 230421C3660000,1,2023-04-21,2022-12-23,C,3660.0,320.6,322.0,...,<NA>,<NA>,False,2022-12-30 16:15:00,2023-04-21 09:30:00,111 days 17:15:00,2023-04-21,1.0,3874.60935,321.3
1713336,108105.0,2022-12-30,SPX 230421C3665000,1,2023-04-21,NaT,C,3665.0,317.1,318.6,...,<NA>,<NA>,False,2022-12-30 16:15:00,2023-04-21 09:30:00,111 days 17:15:00,2023-04-21,1.0,3874.60935,317.85


forming portfolio

In [14]:
#from VIX_SKEW_replication.interest_rate_consol import get_yc_history
#yc_pull=get_yc_history()
#yc_pull
from vix_skew_returns import calculate_interest_rates

In [221]:
#df3[df3.date.isin(third_fridays)].groupby(['date'])['exdate'].min()
#for fri in third_fridays


#portfolio_df={}
portfolio_df=pd.DataFrame()
id_counter=1
for expir in sorted(df4.exdate.unique()):
    print(f'trade_date: {expir}')
    #display(df4[df4.date==expir].exdate.min())
    slice_=df4[df4.date==expir]
    closest_expir=slice_.exdate.min()
    if not pd.isnull(closest_expir):
        print(f'expiration: {closest_expir}')
        
        #PROBLEM: LONGER THAN 45 day options are being pikced up
        term_df=slice_[(slice_.exdate==closest_expir)]
        #display(slice_[(slice_.exdate==closest_expir)] )
        #display(term_df)

        ATM_strike_cands=term_df[~( (pd.isnull(term_df.best_bid)) | (pd.isnull(term_df.best_offer)) ) 
                                & ~(term_df.best_bid>term_df.best_offer) 
                                & ~(term_df.best_bid<=0)   ]
        mins_in_year=365*24*60
        mins_to_expir=ATM_strike_cands.time_to_exp.iloc[0].total_seconds()/60
        t=mins_to_expir/mins_in_year
        #print(ATM_strike_cands.iloc[0].time_to_exp.days)
        days_to_exp=ATM_strike_cands.iloc[0].time_to_exp.days
        #print(calculate_interest_rates(expir,ATM_strike_cands.iloc[0].time_to_exp.days))
        r=calculate_interest_rates(expir,ATM_strike_cands.iloc[0].time_to_exp.days)
        #display(ATM_strike_cands)

        def min_strike_diff(slice):
            if ('C' in slice.cp_flag.unique()) and ('P' in slice.cp_flag.unique()):
                return abs( slice[slice.cp_flag=='P'].midpoint_price.values[0] - slice[slice.cp_flag=='C'].midpoint_price.values[0])
        
        F_strike=ATM_strike_cands.groupby(['strike_price']).apply(min_strike_diff).idxmin()
        #print(F_strike)
        call_put_diff=ATM_strike_cands[(ATM_strike_cands.strike_price==F_strike)].sort_values(by='cp_flag')['midpoint_price'].diff().dropna().values[0]
        F=F_strike+np.exp(r*t)*call_put_diff
        K0=term_df[term_df.strike_price<=F].strike_price.max()
        #print(K0,F)
        #Which Fwd Price to Use
        #print('OptionMetrics Fwd:')
        #print(term_df.forwardprice.unique())
        #print('CBOE Fwd:')
        #print(F)

        def filter_included_options(opt_type):
            if opt_type=='put':
                OOM_opts=term_df[(term_df.strike_price<K0)&(term_df.cp_flag=='P')]
                OOM_opts=OOM_opts.copy()
                OOM_opts.sort_values(by=['strike_price'],ascending=False,inplace=True) #sort upside down for puts
                #print('puts')
                #display(OOM_opts)
                OOM_opts['excl_ind']=OOM_opts.best_bid.apply(lambda x: pd.isnull(x) or x<=0)
                #OOM_opts.sort_values(by=['strike_price'],ascending=False,inplace=True) #sort upside down for puts
            else:
                OOM_opts=term_df[(term_df.strike_price>K0)&(term_df.cp_flag=='C')]
                OOM_opts=OOM_opts.copy()
                OOM_opts.sort_values(by=['strike_price'],ascending=True,inplace=True) #sort upside down for puts
                #print('calls')
                #display(OOM_opts)
                OOM_opts['excl_ind']=OOM_opts.best_bid.apply(lambda x: pd.isnull(x) or x<=0)
                #dont need to change sorting order for calls

            OOM_opts['excl_ind']=OOM_opts['excl_ind'].cumsum()
            #print(opt_type)
            #display(OOM_opts[OOM_opts.excl_ind>=2])
            incl_opts=OOM_opts[OOM_opts.excl_ind<2]
            incl_opts=incl_opts[incl_opts.best_bid>0]
            if opt_type=='put':
                incl_opts.sort_values(by=['strike_price'],ascending=True,inplace=True)#change back
            return incl_opts

        incl_puts,incl_calls=filter_included_options('put'),filter_included_options('call')
        #display(incl_puts)
        #display(incl_calls)
        
        df_K0=term_df[term_df.strike_price==K0]
        #display(df_K0[['optionid','best_bid','best_offer']])
        #print(4/0)
        pca=pd.DataFrame.from_dict({'strike_price':K0,
                                    'cp_flag':'P/C Avg', #put-call average
                                    'midpoint_price':term_df[(term_df.strike_price==K0)]['midpoint_price'].mean(),
                                    'best_bid':None,'best_offer':None,'optionid':None},
                                    orient='index').T

        opt_portfolio=pd.concat([incl_puts,pca,incl_calls])[['strike_price',
                                                            'cp_flag',
                                                            'midpoint_price',
                                                            'best_bid','best_offer',
                                                            'optionid']]
        #display(opt_portfolio)
        #display(opt_portfolio[opt_portfolio.strike_price==K0])
        opt_portfolio['dK']=(opt_portfolio.strike_price.shift(-1)-opt_portfolio.strike_price.shift(1))/2
        opt_portfolio=opt_portfolio.copy()
        #opt_portfolio.iloc[0]['dK']=opt_portfolio.iloc[1]['strike_price']-opt_portfolio.iloc[0]['strike_price']
        #opt_portfolio.iloc[-1]['dK']=opt_portfolio.iloc[-1]['strike_price']-opt_portfolio.iloc[-2]['strike_price']
        opt_portfolio.iloc[0,-1]=opt_portfolio.iloc[1]['strike_price']-opt_portfolio.iloc[0]['strike_price']
        opt_portfolio.iloc[-1,-1]=opt_portfolio.iloc[-1]['strike_price']-opt_portfolio.iloc[-2]['strike_price']
        #display(opt_portfolio)
        

        # . . .

        #display(df_K0)
        
        # VIX and SVIX
        VIX_opt_weight=2*opt_portfolio.dK/(opt_portfolio.strike_price**2)
        SVIX_opt_weight=2*opt_portfolio.dK/(F**2)
        # SKEW
        p1_option_weight=-opt_portfolio.dK/(opt_portfolio.strike_price**2)
        p2_option_weight=2*opt_portfolio.dK/(opt_portfolio.strike_price**2)
        #print( (opt_portfolio.strike_price/F).min() )
        #display(opt_portfolio[pd.isnull(opt_portfolio.strike_price)])
        #print(p2_option_weight)
        #print( 1-np.log(opt_portfolio.strike_price.astype(np.float64)/F) )
        p2_option_weight*=1-np.log(opt_portfolio.strike_price.astype(np.float64)/F)
        p3_option_weight=3*opt_portfolio.dK/(opt_portfolio.strike_price**2)
        p3_option_weight*=2*np.log(opt_portfolio.strike_price.astype(np.float64)/F)-np.log(opt_portfolio.strike_price.astype(np.float64)/F)**2

        e1=-(1+np.log(F/K0)-(F/K0))
        e2=2*np.log(K0/F)*(F/K0-1)+(1/2)*np.log(K0/F)**2
        e3=3*np.log(K0/F)**2 * (1/3*np.log(K0/F)-1+F/K0)
        P1=( np.exp(r*t)*(opt_portfolio.midpoint_price*p1_option_weight) ).sum()
        P1+=e1
        P2=( np.exp(r*t)*(opt_portfolio.midpoint_price*p2_option_weight) ).sum()
        P2+=e2
        P3=( np.exp(r*t)*(opt_portfolio.midpoint_price*p3_option_weight) ).sum()
        P3+=e3
        sigma=np.sqrt(P2-P1**2)
        
        opt_portfolio['VIX_opt_weight']=VIX_opt_weight*np.exp(r*t)
        opt_portfolio['SVIX_opt_weight']=SVIX_opt_weight*np.exp(r*t)
        opt_portfolio['meth1_opt_weight']=(p3_option_weight - 3*P1*p2_option_weight + 2*P1**2*p1_option_weight) * np.exp(r*t)/sigma**3
        opt_portfolio['meth2_opt_weight']=(-6*p1_option_weight - 3*p2_option_weight) * np.exp(r*t)/sigma**3
        opt_portfolio['meth3_opt_weight']=(SVIX_opt_weight-VIX_opt_weight) * 3/2*np.exp(r*t)/sigma**3
        #display(opt_portfolio)
        #print( (opt_portfolio.VIX_opt_weight*opt_portfolio.midpoint_price).sum() )
        #print( (opt_portfolio.SVIX_opt_weight*opt_portfolio.midpoint_price).sum() )
        #print( (opt_portfolio.meth1_opt_weight*opt_portfolio.midpoint_price).sum() )
        #print( (opt_portfolio.meth2_opt_weight*opt_portfolio.midpoint_price).sum() )
        #print( (opt_portfolio.meth3_opt_weight*opt_portfolio.midpoint_price).sum() )
        
        # at the end drop all the uneeded columsna and append the K0 cols at the end with 50/50 weight
        #print(opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].VIX_opt_weight)
        #print(opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].SVIX_opt_weight)
        #print(opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].dK)
        df_K0=df_K0[['strike_price','cp_flag','midpoint_price','best_bid','best_offer','optionid']]
        df_K0['dK']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].dK.values[0]
        #split the weight 50-50 for the ATM call and put
        df_K0['VIX_opt_weight']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].VIX_opt_weight.values[0]*1/2
        df_K0['SVIX_opt_weight']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].SVIX_opt_weight.values[0]*1/2
        df_K0['meth1_opt_weight']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].meth1_opt_weight.values[0]*1/2
        df_K0['meth2_opt_weight']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].meth2_opt_weight.values[0]*1/2
        df_K0['meth3_opt_weight']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].meth3_opt_weight.values[0]*1/2
        opt_portfolio=opt_portfolio[opt_portfolio.cp_flag!='P/C Avg']

        opt_portfolio_2=pd.concat([opt_portfolio,df_K0])[['strike_price',
                                                            'cp_flag',
                                                            'midpoint_price',
                                                            'best_bid','best_offer',
                                                            'optionid',
                                                            'VIX_opt_weight',
                                                            'SVIX_opt_weight',
                                                            'meth1_opt_weight',
                                                            'meth2_opt_weight',
                                                            'meth3_opt_weight']]
        display(opt_portfolio_2)
        opt_portfolio_2['portfolio_id']=id_counter
        #display(opt_portfolio_2.iloc[-1].VIX_opt_weight)
        opt_portfolio_2['VIX_trade_price']=opt_portfolio_2.apply(lambda x: x.best_offer if x.VIX_opt_weight >0 else x.best_bid,axis=1)
        opt_portfolio_2['SVIX_trade_price']=opt_portfolio_2.apply(lambda x: x.best_offer if x.SVIX_opt_weight >0 else x.best_bid,axis=1)
        opt_portfolio_2['meth1_trade_price']=opt_portfolio_2.apply(lambda x: x.best_offer if x.meth1_opt_weight >0 else x.best_bid,axis=1)
        opt_portfolio_2['meth2_trade_price']=opt_portfolio_2.apply(lambda x: x.best_offer if x.meth2_opt_weight >0 else x.best_bid,axis=1)
        opt_portfolio_2['meth3_trade_price']=opt_portfolio_2.apply(lambda x: x.best_offer if x.meth3_opt_weight >0 else x.best_bid,axis=1)

        opt_portfolio_2['trade_date']=expir
        opt_portfolio_2['portfolio_expiration']=closest_expir
        opt_portfolio_2['days_to_exp']=days_to_exp
        #this is cutting off the rounding
        #opt_portfolio_2.sort_values(by=['optionid'],inplace=True)
        #opt_portfolio_2=opt_portfolio_2.sort_values(by=['optionid'])
        

        #print( (opt_portfolio_2.VIX_opt_weight*opt_portfolio_2.midpoint_price).sum() )
        #print( (opt_portfolio_2.SVIX_opt_weight*opt_portfolio_2.midpoint_price).sum() )
        #display(opt_portfolio_2)
        #then append to final
        portfolio_df=pd.concat([portfolio_df, opt_portfolio_2], ignore_index=True)

        #get a dict of optionIDs and weights
        #for K0 need 50-50 on call and put, same w bid and ask

        #NEXT BLOCK
        id_counter+=1

        #maybe down here we can do the prices of the portfolios
        print('~~~~~~~~~~~~~~')
        #display(opt_portfolio_2.days_to_exp.unique() )
        #print(  len( opt_portfolio_2.optionid.unique().tolist() ) )
        z = df4[ (df4.optionid.isin(opt_portfolio_2.optionid.unique().tolist()))
                & (df4.date>opt_portfolio_2.trade_date.values[0])]
        #& > trade_date
        #display(z ) 
        print('~~~~~~~~~~~~~~')
        #groupby date.apply(portfolio_function)
        def portfolio_func(sub_slice):
            sub_slice.sort_values(by=['optionid'],inplace=True)
            #display(sub_slice)
            #display(opt_portfolio_2.iloc[0])
            #display(opt_portfolio_2)
            #print( {int(k):v for k,v in zip(opt_portfolio_2.optionid.values,opt_portfolio_2.VIX_opt_weight.values)} )
            dict1={int(k):v for k,v in zip(opt_portfolio_2.optionid.values,opt_portfolio_2.VIX_opt_weight.values)}
            dict2={int(k):v for k,v in zip(sub_slice.optionid.values,sub_slice.midpoint_price.values)}
            #print( {int(k):v for k,v in zip(sub_slice.optionid.values,sub_slice.midpoint_price.values)} )
            dict3={key: dict1.get(key, 0) * dict2.get(key, 0) for key in set(dict1) | set(dict2)}
            print(len(dict3.keys()))
            print( sum(dict3.values()) )
            #print( (sub_slice.midpoint_price*opt_portfolio_2.VIX_opt_weight).sum() )
            

            print(3/0)
        z.groupby(['date']).apply(portfolio_func)
        #print(4/0)

    print('-------------')
    #print(5/0)
    #if expir not in third_fridays:
    #    print(expir)


portfolio_df

trade_date: 2022-01-21 00:00:00
expiration: 2022-02-18 00:00:00


/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_31494/1847669386.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  F_strike=ATM_strike_cands.groupby(['strike_price']).apply(min_strike_diff).idxmin()
/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_31494/1847669386.py:89: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  opt_portfolio=pd.concat([incl_puts,pca,incl_calls])[['strike_price',


,strike_price,cp_flag,midpoint_price,best_bid,best_offer,optionid,VIX_opt_weight,SVIX_opt_weight,meth1_opt_weight,meth2_opt_weight,meth3_opt_weight
100517,1600.0,P,0.15,0.05,0.25,139180003.0,0.000078,0.00001,-0.593478,-0.395463,-0.169307
100518,1700.0,P,0.2,0.1,0.3,139180004.0,0.000069,0.00001,-0.484237,-0.329368,-0.147047
100519,1800.0,P,0.25,0.15,0.35,139180005.0,0.000062,0.00001,-0.398087,-0.27618,-0.128392
100520,1900.0,P,0.3,0.2,0.4,139180006.0,0.000055,0.00001,-0.329387,-0.232924,-0.112605
100522,2000.0,P,0.375,0.3,0.45,139180008.0,0.000038,0.000008,-0.205545,-0.148061,-0.074345
...,...,...,...,...,...,...,...,...,...,...,...
100489,5300.0,C,0.1,0.05,0.15,139179986.0,0.000002,0.000003,0.001503,0.001629,0.001968
100490,5325.0,C,0.125,0.05,0.2,143493697.0,0.000002,0.000003,0.001523,0.001656,0.00201
100491,5350.0,C,0.125,0.05,0.2,141848457.0,0.000002,0.000003,0.001542,0.001681,0.002051
104976,4410.0,C,117.5,116.9,118.1,142041464.0,0.0,0.0,0.000004,-0.000001,-0.000001


~~~~~~~~~~~~~~
~~~~~~~~~~~~~~
378
0.006323628930973683


ZeroDivisionError: division by zero

In [177]:
#z=df4.groupby(['optionid'])['exdate'].nunique()
#z[z>=2]

y=df4.groupby(['date','exdate','strike_price','cp_flag'])['optionid'].nunique()
y[y>=2]

Series([], Name: optionid, dtype: int64)

In [43]:
#df3[df3.date.isin(third_fridays)].groupby(['date'])['exdate'].min()
#for fri in third_fridays


portfolio_dict={}
portfolio_df=pd.DataFrame()
id_counter=1
for expir in sorted(df4.exdate.unique()):
    print(f'trade_date: {expir}')
    #display(df4[df4.date==expir].exdate.min())
    slice_=df4[df4.date==expir]
    next_expir = pd.Series(slice_.exdate.unique()).nsmallest(3)
    #portfolio_dict[expir]={}
    portfolio_dict[id_counter]={'Trade Date':expir}

    if not pd.isna(next_expir).all():
        #for term in next_expir:
        for i,term in enumerate(next_expir):
            #portfolio_dict[expir][f'{i+1} month expiration']=term
            portfolio_dict[id_counter][f'{i+1} month expiration']=term
            #print(term)
            term_df=slice_[(slice_.exdate==term)]
            #display(term_df)
            #print(3/0)
            ATM_strike_cands=term_df[~( (pd.isnull(term_df.best_bid)) | (pd.isnull(term_df.best_offer)) ) 
                        & ~(term_df.best_bid>term_df.best_offer) 
                        & ~(term_df.best_bid<=0)   ]
            
            mins_in_year=365*24*60
            mins_to_expir=ATM_strike_cands.time_to_exp.iloc[0].total_seconds()/60
            t=mins_to_expir/mins_in_year
            #print(ATM_strike_cands.iloc[0].time_to_exp.days)
            days_to_exp=ATM_strike_cands.iloc[0].time_to_exp.days
            #print(calculate_interest_rates(expir,ATM_strike_cands.iloc[0].time_to_exp.days))
            print(f'{ATM_strike_cands.iloc[0].time_to_exp.days} days to expiration')
            r=calculate_interest_rates(expir,ATM_strike_cands.iloc[0].time_to_exp.days)
            #missing data in yield curve fiel was throwing error
            #print(r)
            #print('~~~~~~~~~~~~~~~~~~')
            #display(ATM_strike_cands)

            def min_strike_diff(slice):
                if ('C' in slice.cp_flag.unique()) and ('P' in slice.cp_flag.unique()):
                    return abs( slice[slice.cp_flag=='P'].midpoint_price.values[0] - slice[slice.cp_flag=='C'].midpoint_price.values[0])
            
            F_strike=ATM_strike_cands.groupby(['strike_price']).apply(min_strike_diff).idxmin()
            #print(F_strike)
            call_put_diff=ATM_strike_cands[(ATM_strike_cands.strike_price==F_strike)].sort_values(by='cp_flag')['midpoint_price'].diff().dropna().values[0]
            F=F_strike+np.exp(r*t)*call_put_diff
            K0=term_df[term_df.strike_price<=F].strike_price.max()
            #print(K0,F)
            #Which Fwd Price to Use
            #print('OptionMetrics Fwd:')
            #print(term_df.forwardprice.unique())
            #print('CBOE Fwd:')
            #print(F)

            def filter_included_options(opt_type):
                if opt_type=='put':
                    OOM_opts=term_df[(term_df.strike_price<K0)&(term_df.cp_flag=='P')]
                    OOM_opts=OOM_opts.copy()
                    #OOM_opts.sort_values(by=['strike_price'],ascending=False,inplace=True) #sort upside down for puts
                    #print('puts')
                    #display(OOM_opts)
                    OOM_opts['excl_ind']=OOM_opts.best_bid.apply(lambda x: pd.isnull(x) or x<=0)
                    OOM_opts.sort_values(by=['strike_price'],ascending=False,inplace=True) #sort upside down for puts
                else:
                    OOM_opts=term_df[(term_df.strike_price>K0)&(term_df.cp_flag=='C')]
                    OOM_opts=OOM_opts.copy()
                    #OOM_opts.sort_values(by=['strike_price'],ascending=True,inplace=True) #sort upside down for puts
                    #print('calls')
                    #display(OOM_opts)
                    OOM_opts['excl_ind']=OOM_opts.best_bid.apply(lambda x: pd.isnull(x) or x<=0)
                    #dont need to change sorting order for calls

                OOM_opts['excl_ind']=OOM_opts['excl_ind'].cumsum()
                #print(opt_type)
                #display(OOM_opts[OOM_opts.excl_ind>=2])
                incl_opts=OOM_opts[OOM_opts.excl_ind<2]
                incl_opts=incl_opts[incl_opts.best_bid>0]
                if opt_type=='put':
                    incl_opts.sort_values(by=['strike_price'],ascending=True,inplace=True)#change back
                return incl_opts

            incl_puts,incl_calls=filter_included_options('put'),filter_included_options('call')
            #display(incl_puts)
            #display(incl_calls)
            
            df_K0=term_df[term_df.strike_price==K0]
            #display(df_K0[['optionid','best_bid','best_offer']])
            #print(4/0)
            pca=pd.DataFrame.from_dict({'strike_price':K0,
                                        'cp_flag':'P/C Avg', #put-call average
                                        'midpoint_price':term_df[(term_df.strike_price==K0)]['midpoint_price'].mean(),
                                        'best_bid':None,'best_offer':None,'optionid':None},
                                        orient='index').T

            opt_portfolio=pd.concat([incl_puts,pca,incl_calls])[['strike_price',
                                                                'cp_flag',
                                                                'midpoint_price',
                                                                'best_bid','best_offer',
                                                                'optionid']]
            #display(opt_portfolio)
            #display(opt_portfolio[opt_portfolio.strike_price==K0])
            opt_portfolio['dK']=(opt_portfolio.strike_price.shift(-1)-opt_portfolio.strike_price.shift(1))/2
            opt_portfolio=opt_portfolio.copy()
            #opt_portfolio.iloc[0]['dK']=opt_portfolio.iloc[1]['strike_price']-opt_portfolio.iloc[0]['strike_price']
            #opt_portfolio.iloc[-1]['dK']=opt_portfolio.iloc[-1]['strike_price']-opt_portfolio.iloc[-2]['strike_price']
            opt_portfolio.iloc[0,-1]=opt_portfolio.iloc[1]['strike_price']-opt_portfolio.iloc[0]['strike_price']
            opt_portfolio.iloc[-1,-1]=opt_portfolio.iloc[-1]['strike_price']-opt_portfolio.iloc[-2]['strike_price']
            #display(opt_portfolio)
            
            # VIX and SVIX
            #addign 2/t for now instead of 2
            VIX_opt_weight=(2/t)*opt_portfolio.dK/(opt_portfolio.strike_price**2)
            SVIX_opt_weight=(2/t)*opt_portfolio.dK/(F**2)
            # SKEW
            p1_option_weight=-opt_portfolio.dK/(opt_portfolio.strike_price**2)
            p2_option_weight=2*opt_portfolio.dK/(opt_portfolio.strike_price**2)
            #print( (opt_portfolio.strike_price/F).min() )
            #display(opt_portfolio[pd.isnull(opt_portfolio.strike_price)])
            #print(p2_option_weight)
            #print( 1-np.log(opt_portfolio.strike_price.astype(np.float64)/F) )
            p2_option_weight*=1-np.log(opt_portfolio.strike_price.astype(np.float64)/F)
            p3_option_weight=3*opt_portfolio.dK/(opt_portfolio.strike_price**2)
            p3_option_weight*=2*np.log(opt_portfolio.strike_price.astype(np.float64)/F)-np.log(opt_portfolio.strike_price.astype(np.float64)/F)**2

            e1=-(1+np.log(F/K0)-(F/K0))
            e2=2*np.log(K0/F)*(F/K0-1)+(1/2)*np.log(K0/F)**2
            e3=3*np.log(K0/F)**2 * (1/3*np.log(K0/F)-1+F/K0)
            P1=( np.exp(r*t)*(opt_portfolio.midpoint_price*p1_option_weight) ).sum()
            P1+=e1
            P2=( np.exp(r*t)*(opt_portfolio.midpoint_price*p2_option_weight) ).sum()
            P2+=e2
            P3=( np.exp(r*t)*(opt_portfolio.midpoint_price*p3_option_weight) ).sum()
            P3+=e3
            sigma=np.sqrt(P2-P1**2)
            
            opt_portfolio['VIX_opt_weight']=VIX_opt_weight*np.exp(r*t)
            opt_portfolio['SVIX_opt_weight']=SVIX_opt_weight*np.exp(r*t)
            opt_portfolio['meth1_opt_weight']=(p3_option_weight - 3*P1*p2_option_weight + 2*P1**2*p1_option_weight) * np.exp(r*t)/sigma**3
            opt_portfolio['meth2_opt_weight']=(-6*p1_option_weight - 3*p2_option_weight) * np.exp(r*t)/sigma**3
            opt_portfolio['meth3_opt_weight']=(SVIX_opt_weight-VIX_opt_weight) * 3/2*np.exp(r*t)/sigma**3
            #display(opt_portfolio)
            print( np.sqrt((opt_portfolio.VIX_opt_weight*opt_portfolio.midpoint_price).sum())*100 )
            print( np.sqrt((opt_portfolio.SVIX_opt_weight*opt_portfolio.midpoint_price).sum())*100 )
            #print( (opt_portfolio.VIX_opt_weight*opt_portfolio.midpoint_price).sum() )
            #print( (opt_portfolio.SVIX_opt_weight*opt_portfolio.midpoint_price).sum() )
            #print( 100-10*(opt_portfolio.meth1_opt_weight*opt_portfolio.midpoint_price).sum() )
            #print( 100-10*(opt_portfolio.meth2_opt_weight*opt_portfolio.midpoint_price).sum() )
            #print( 100-10*(opt_portfolio.meth3_opt_weight*opt_portfolio.midpoint_price).sum() )
            #print( (opt_portfolio.meth1_opt_weight*opt_portfolio.midpoint_price).sum() )
            #print( (opt_portfolio.meth2_opt_weight*opt_portfolio.midpoint_price).sum() )
            #print( (opt_portfolio.meth3_opt_weight*opt_portfolio.midpoint_price).sum() )
            
            #portfolio_dict[expir][f'{i+1} month VIX']=np.sqrt((opt_portfolio.VIX_opt_weight*opt_portfolio.midpoint_price).sum())*100
            #portfolio_dict[expir][f'{i+1} month SVIX']=np.sqrt((opt_portfolio.SVIX_opt_weight*opt_portfolio.midpoint_price).sum())*100
            portfolio_dict[id_counter][f'{i+1} month VIX']=np.sqrt((opt_portfolio.VIX_opt_weight*opt_portfolio.midpoint_price).sum())*100
            portfolio_dict[id_counter][f'{i+1} month SVIX']=np.sqrt((opt_portfolio.SVIX_opt_weight*opt_portfolio.midpoint_price).sum())*100

            """
            # at the end drop all the uneeded columsna and append the K0 cols at the end with 50/50 weight
            #print(opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].VIX_opt_weight)
            #print(opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].SVIX_opt_weight)
            #print(opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].dK)
            df_K0=df_K0[['strike_price','cp_flag','midpoint_price','best_bid','best_offer','optionid']]
            df_K0['dK']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].dK.values[0]
            #split the weight 50-50 for the ATM call and put
            df_K0['VIX_opt_weight']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].VIX_opt_weight.values[0]*1/2
            df_K0['SVIX_opt_weight']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].SVIX_opt_weight.values[0]*1/2
            df_K0['meth1_opt_weight']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].meth1_opt_weight.values[0]*1/2
            df_K0['meth2_opt_weight']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].meth2_opt_weight.values[0]*1/2
            df_K0['meth3_opt_weight']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].meth3_opt_weight.values[0]*1/2
            
            opt_portfolio=opt_portfolio[opt_portfolio.cp_flag!='P/C Avg']

            opt_portfolio_2=pd.concat([opt_portfolio,df_K0])[['strike_price',
                                                                'cp_flag',
                                                                'midpoint_price',
                                                                'best_bid','best_offer',
                                                                'optionid',
                                                                'VIX_opt_weight',
                                                                'SVIX_opt_weight',
                                                                'meth1_opt_weight',
                                                                'meth2_opt_weight',
                                                                'meth3_opt_weight']]
            #display(opt_portfolio_2)
            opt_portfolio_2['portfolio_id']=id_counter
            #display(opt_portfolio_2.iloc[-1].VIX_opt_weight)
            opt_portfolio_2['VIX_trade_price']=opt_portfolio_2.apply(lambda x: x.best_offer if x.VIX_opt_weight >0 else x.best_bid,axis=1)
            opt_portfolio_2['SVIX_trade_price']=opt_portfolio_2.apply(lambda x: x.best_offer if x.SVIX_opt_weight >0 else x.best_bid,axis=1)
            opt_portfolio_2['meth1_trade_price']=opt_portfolio_2.apply(lambda x: x.best_offer if x.meth1_opt_weight >0 else x.best_bid,axis=1)
            opt_portfolio_2['meth2_trade_price']=opt_portfolio_2.apply(lambda x: x.best_offer if x.meth2_opt_weight >0 else x.best_bid,axis=1)
            opt_portfolio_2['meth3_trade_price']=opt_portfolio_2.apply(lambda x: x.best_offer if x.meth3_opt_weight >0 else x.best_bid,axis=1)

            opt_portfolio_2['trade_date']=expir
            opt_portfolio_2['portfolio_expiration']=closest_expir
            opt_portfolio_2['days_to_exp']=days_to_exp
            #this is cutting off the rounding
            #opt_portfolio_2.sort_values(by=['optionid'],inplace=True)
            #opt_portfolio_2=opt_portfolio_2.sort_values(by=['optionid'])
            

            print( (opt_portfolio_2.VIX_opt_weight*opt_portfolio_2.midpoint_price).sum() )
            print( (opt_portfolio_2.SVIX_opt_weight*opt_portfolio_2.midpoint_price).sum() )
            display(opt_portfolio_2)
            """
        id_counter+=1
            
    
    print('---------')



#portfolio_df
portfolio_dict

trade_date: 2022-01-21 00:00:00
27 days to expiration


/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_17480/2215257049.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  F_strike=ATM_strike_cands.groupby(['strike_price']).apply(min_strike_diff).idxmin()
/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_17480/2215257049.py:98: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  opt_portfolio=pd.concat([incl_puts,pca,incl_calls])[['strike_price',


28.508949149985224
26.316567045861955
55 days to expiration


/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_17480/2215257049.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  F_strike=ATM_strike_cands.groupby(['strike_price']).apply(min_strike_diff).idxmin()
/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_17480/2215257049.py:98: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  opt_portfolio=pd.concat([incl_puts,pca,incl_calls])[['strike_price',


29.360311492593773
26.10321979798716
82 days to expiration


...

/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_17480/2215257049.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  F_strike=ATM_strike_cands.groupby(['strike_price']).apply(min_strike_diff).idxmin()
/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_17480/2215257049.py:98: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  opt_portfolio=pd.concat([incl_puts,pca,incl_calls])[['strike_price',


25.173285656237443
23.923823898010134
90 days to expiration


/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_17480/2215257049.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  F_strike=ATM_strike_cands.groupby(['strike_price']).apply(min_strike_diff).idxmin()
/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_17480/2215257049.py:98: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  opt_portfolio=pd.concat([incl_puts,pca,incl_calls])[['strike_price',


25.02705921482494
23.15834265903122
---------
trade_date: 2023-01-20 00:00:00
---------
trade_date: 2023-02-17 00:00:00
---------
trade_date: 2023-03-17 00:00:00
---------
trade_date: 2023-04-21 00:00:00
---------
trade_date: 2023-05-19 00:00:00
---------
trade_date: 2023-06-16 00:00:00
---------
trade_date: 2023-07-21 00:00:00
---------
trade_date: 2023-08-18 00:00:00
---------
trade_date: 2023-09-15 00:00:00
---------
trade_date: 2023-10-20 00:00:00
---------
trade_date: 2023-11-17 00:00:00
---------
trade_date: 2023-12-15 00:00:00
---------
trade_date: 2024-01-19 00:00:00
---------
trade_date: 2024-03-15 00:00:00
---------
trade_date: 2024-06-21 00:00:00
---------
trade_date: 2024-12-20 00:00:00
---------
trade_date: 2025-12-19 00:00:00
---------
trade_date: 2026-12-18 00:00:00
---------
trade_date: 2027-12-17 00:00:00
---------


{1: {'Trade Date': Timestamp('2022-01-21 00:00:00'),
  '1 month expiration': Timestamp('2022-02-18 00:00:00'),
  '1 month VIX': np.float64(28.508949149985224),
  '1 month SVIX': np.float64(26.316567045861955),
  '2 month expiration': Timestamp('2022-03-18 00:00:00'),
  '2 month VIX': np.float64(29.360311492593773),
  '2 month SVIX': np.float64(26.10321979798716),
  '3 month expiration': Timestamp('2022-04-14 00:00:00'),
  '3 month VIX': np.float64(29.529385013310378),
  '3 month SVIX': np.float64(25.50145233896075)},
 2: {'Trade Date': Timestamp('2022-02-18 00:00:00'),
  '1 month expiration': Timestamp('2022-03-18 00:00:00'),
  '1 month VIX': np.float64(27.944852872686397),
  '1 month SVIX': np.float64(26.310594357848714),
  '2 month expiration': Timestamp('2022-04-14 00:00:00'),
  '2 month VIX': np.float64(28.871975779438763),
  '2 month SVIX': np.float64(25.9434314477604),
  '3 month expiration': Timestamp('2022-05-20 00:00:00'),
  '3 month VIX': np.float64(32.267508468349334),
  '3 

In [44]:
pd.DataFrame.from_dict(portfolio_dict).T

,Trade Date,1 month expiration,1 month VIX,1 month SVIX,2 month expiration,2 month VIX,2 month SVIX,3 month expiration,3 month VIX,3 month SVIX
1,2022-01-21 00:00:00,2022-02-18 00:00:00,28.508949,26.316567,2022-03-18 00:00:00,29.360311,26.10322,2022-04-14 00:00:00,29.529385,25.501452
2,2022-02-18 00:00:00,2022-03-18 00:00:00,27.944853,26.310594,2022-04-14 00:00:00,28.871976,25.943431,2022-05-20 00:00:00,32.267508,28.309483
3,2022-03-18 00:00:00,2022-04-14 00:00:00,24.153464,22.710039,2022-05-20 00:00:00,25.631451,22.807147,2022-06-17 00:00:00,26.700805,23.002322
4,2022-04-14 00:00:00,2022-05-20 00:00:00,22.80288,21.312417,2022-06-17 00:00:00,24.700094,22.177427,2022-07-15 00:00:00,25.929275,22.597396
5,2022-05-20 00:00:00,2022-06-17 00:00:00,32.353925,30.910909,2022-07-15 00:00:00,30.072215,27.552132,2022-08-19 00:00:00,31.161096,27.41904
6,2022-06-17 00:00:00,2022-07-15 00:00:00,30.895831,29.563328,2022-08-19 00:00:00,31.857452,29.058309,2022-09-16 00:00:00,27.937712,22.645802
7,2022-07-15 00:00:00,2022-08-19 00:00:00,20.222749,18.647734,2022-09-16 00:00:00,26.572762,24.76407,2022-10-21 00:00:00,27.141292,24.41048
8,2022-08-19 00:00:00,2022-09-16 00:00:00,19.920342,19.187919,2022-10-21 00:00:00,22.680372,21.175158,2022-11-18 00:00:00,24.229264,22.178285
9,2022-09-16 00:00:00,2022-10-21 00:00:00,26.005189,24.968231,2022-11-18 00:00:00,27.185321,25.440845,2022-12-16 00:00:00,27.598201,25.266765
10,2022-10-21 00:00:00,2022-11-18 00:00:00,30.25106,29.351941,2022-12-16 00:00:00,30.423157,28.73936,2023-01-20 00:00:00,30.686663,27.918664


In [60]:
df4[df4.optionid==143178031.0]

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,...,root,suffix,holiday_exp,datetime_close,ex_time,time_to_exp,expiration,amsettlement,forwardprice,oom
1076,108105.0,2022-01-03,SPX 220218C4535000,1,2022-02-18,2022-01-03,C,4535.0,288.7,289.8,...,<NA>,<NA>,False,2022-01-03 16:15:00,2022-02-18 09:30:00,45 days 17:15:00,2022-02-18,1.0,4787.149658,False
7986,108105.0,2022-01-04,SPX 220218C4535000,1,2022-02-18,2022-01-03,C,4535.0,292.2,293.2,...,<NA>,<NA>,False,2022-01-04 16:15:00,2022-02-18 09:30:00,44 days 17:15:00,2022-02-18,1.0,4791.890805,False
14412,108105.0,2022-01-05,SPX 220218C4535000,1,2022-02-18,2022-01-03,C,4535.0,224.7,226.0,...,<NA>,<NA>,False,2022-01-05 16:15:00,2022-02-18 09:30:00,43 days 17:15:00,2022-02-18,1.0,4700.183347,False
24562,108105.0,2022-01-06,SPX 220218C4535000,1,2022-02-18,2022-01-03,C,4535.0,217.6,218.8,...,<NA>,<NA>,False,2022-01-06 16:15:00,2022-02-18 09:30:00,42 days 17:15:00,2022-02-18,1.0,4691.907404,False
30589,108105.0,2022-01-07,SPX 220218C4535000,1,2022-02-18,2022-01-03,C,4535.0,200.0,201.1,...,<NA>,<NA>,False,2022-01-07 16:15:00,2022-02-18 09:30:00,41 days 17:15:00,2022-02-18,1.0,4675.783199,False
40994,108105.0,2022-01-10,SPX 220218C4535000,1,2022-02-18,2022-01-10,C,4535.0,187.7,188.9,...,<NA>,<NA>,False,2022-01-10 16:15:00,2022-02-18 09:30:00,38 days 17:15:00,2022-02-18,1.0,4663.954936,False
46418,108105.0,2022-01-11,SPX 220218C4535000,1,2022-02-18,2022-01-10,C,4535.0,218.0,219.3,...,<NA>,<NA>,False,2022-01-11 16:15:00,2022-02-18 09:30:00,37 days 17:15:00,2022-02-18,1.0,4707.237271,False
53809,108105.0,2022-01-12,SPX 220218C4535000,1,2022-02-18,2022-01-10,C,4535.0,226.6,231.0,...,<NA>,<NA>,False,2022-01-12 16:15:00,2022-02-18 09:30:00,36 days 17:15:00,2022-02-18,1.0,4722.89827,False
64384,108105.0,2022-01-13,SPX 220218C4535000,1,2022-02-18,2022-01-10,C,4535.0,180.7,181.8,...,<NA>,<NA>,False,2022-01-13 16:15:00,2022-02-18 09:30:00,35 days 17:15:00,2022-02-18,1.0,4652.57493,False
67429,108105.0,2022-01-14,SPX 220218C4535000,1,2022-02-18,2022-01-14,C,4535.0,181.1,182.3,...,<NA>,<NA>,False,2022-01-14 16:15:00,2022-02-18 09:30:00,34 days 17:15:00,2022-02-18,1.0,4661.06954,False


In [ ]:
def bid_ask_analysis(threshold=None):

    def skew_weights_1(slice):
        mins_30_days=30*60*24
        slice1=slice[slice.TERM=='near']
        slice2=slice[slice.TERM=='next']
        w=(slice2.MIN_TO_EXPIRE.unique()[0]-mins_30_days)
        w/=(slice2.MIN_TO_EXPIRE.unique()[0]-slice1.MIN_TO_EXPIRE.unique()[0])
        return [w,1-w]

    def portfolio_constructors(slice):

        F=slice.FORWARD_PRICE.unique()[0]
        K0=term_df[term_df.strike_price<=F].strike_price.max()
        K0=slice.K0_SELECTION.unique()[0]
        calls=slice[slice.STRIKE_PRICE>K0]
        puts=slice[slice.STRIKE_PRICE<K0]
        #display(slice)
        #print(5/0)
        pca=pd.DataFrame.from_dict({'STRIKE_PRICE':K0,
                                'CALL_PUT_FLAG':'P/C Avg', #put-call average
                                'midprice':slice[(slice.STRIKE_PRICE==K0)]['midprice'].mean(),
                                'INTEREST_RATE':slice.INTEREST_RATE,

                                'BID_PRICE':slice.BID_PRICE,
                                'ASK_PRICE':slice.ASK_PRICE,
                                'term_weight':slice.term_weight,

                                'YEARS_TO_EXPIRE':slice.YEARS_TO_EXPIRE} ).head(1)
        opt_portfolio=pd.concat([puts,pca,calls],ignore_index=True)[['STRIKE_PRICE','CALL_PUT_FLAG','midprice','INTEREST_RATE','BID_PRICE','ASK_PRICE','term_weight','YEARS_TO_EXPIRE']]
        opt_portfolio['dK']=(opt_portfolio.STRIKE_PRICE.shift(-1)-opt_portfolio.STRIKE_PRICE.shift(1))/2
        opt_portfolio.loc[0,'dK']=opt_portfolio.iloc[1]['STRIKE_PRICE']-opt_portfolio.iloc[0]['STRIKE_PRICE']
        opt_portfolio.loc[opt_portfolio.index[-1],'dK']=opt_portfolio.iloc[-1]['STRIKE_PRICE']-opt_portfolio.iloc[-2]['STRIKE_PRICE']

        #Truncating the portfolio
        #opt_portfolio['moneyness_pct']=F/opt_portfolio['STRIKE_PRICE']-1
        opt_portfolio['moneyness_pct']=opt_portfolio['STRIKE_PRICE']/F-1
        opt_portfolio['z_moneyness']=(opt_portfolio['moneyness_pct']-opt_portfolio['moneyness_pct'].mean()) / opt_portfolio['moneyness_pct'].std() 
        truncation_condition=None
        if truncation_condition:
            opt_portfolio=opt_portfolio.query(truncation_condition)

        if threshold:
            opt_portfolio['BID_PRICE']=np.where(opt_portfolio['moneyness_pct']<=threshold,opt_portfolio['midprice'],opt_portfolio['BID_PRICE'])
            opt_portfolio['ASK_PRICE']=np.where(opt_portfolio['moneyness_pct']<=threshold,opt_portfolio['midprice'],opt_portfolio['ASK_PRICE'])
            
        #print('--------------')
        #display(opt_portfolio)
        #print(5/0)
        r,t=opt_portfolio.INTEREST_RATE.unique()[0],opt_portfolio.YEARS_TO_EXPIRE.unique()[0]
        #display(opt_portfolio)
        #CBOE SKEW
        e1=-(1+np.log(F/K0)-(F/K0))
        e2=2*np.log(K0/F)*(F/K0-1)+(1/2)*np.log(K0/F)**2
        e3=3*np.log(K0/F)**2 * (1/3*np.log(K0/F)-1+F/K0)
        p1_option_weight=-opt_portfolio.dK/(opt_portfolio.STRIKE_PRICE**2)
        p2_option_weight=2*opt_portfolio.dK/(opt_portfolio.STRIKE_PRICE**2)
        p2_option_weight*=1-np.log(opt_portfolio.STRIKE_PRICE/F)
        p3_option_weight=3*opt_portfolio.dK/(opt_portfolio.STRIKE_PRICE**2)
        p3_option_weight*=2*np.log(opt_portfolio.STRIKE_PRICE/F)-np.log(opt_portfolio.STRIKE_PRICE/F)**2

        P1=( np.exp(r*t)*(opt_portfolio.midprice*p1_option_weight) ).sum()
        P1+=e1
        P2=( np.exp(r*t)*(opt_portfolio.midprice*p2_option_weight) ).sum()
        P2+=e2
        P3=( np.exp(r*t)*(opt_portfolio.midprice*p3_option_weight) ).sum()
        P3+=e3

        S_method1=P3-3*P1*P2+2*P1**3
        sigma=np.sqrt(P2-P1**2)
        S_method1/=sigma**3

        #METHOD2
        S_method2=-6*P1 - 3*P2
        S_method2/=sigma**3

        #METHOD3
        #correction_term=(1/t)*(F/K0-1)**2
        VIX_opt_weight=2*opt_portfolio.dK/(opt_portfolio.STRIKE_PRICE**2)
        SVIX_opt_weight=2*opt_portfolio.dK/(F**2)
        VIX_square=( np.exp(r*t)*(opt_portfolio.midprice*VIX_opt_weight) ).sum()
        SVIX_square=( np.exp(r*t)*(opt_portfolio.midprice*SVIX_opt_weight) ).sum()
        #VIX_square+=correction_term
        S_method3=3/2*(SVIX_square-VIX_square)
        S_method3/= sigma**3
        #loop through strikes, if weight pos give low bid if weight neg give high bid

        opt_portfolio['meth1_opt_weight']=(p3_option_weight - 3*P1*p2_option_weight + 2*P1**2*p1_option_weight) * np.exp(r*t)/sigma**3
        opt_portfolio['meth1_opt_weight']*=opt_portfolio['term_weight']
        opt_portfolio['meth2_opt_weight']=(-6*p1_option_weight - 3*p2_option_weight) * np.exp(r*t)/sigma**3
        opt_portfolio['meth2_opt_weight']*=opt_portfolio['term_weight']
        opt_portfolio['meth3_opt_weight']=(SVIX_opt_weight-VIX_opt_weight) * 3/2*np.exp(r*t)/sigma**3
        opt_portfolio['meth3_opt_weight']*=opt_portfolio['term_weight']
        
        opt_portfolio['meth1_bid_contribtion']=opt_portfolio.apply(lambda x: x.meth1_opt_weight*x.BID_PRICE if x.meth1_opt_weight >0 else x.meth1_opt_weight*x.ASK_PRICE,axis=1)
        opt_portfolio['meth1_ask_contribtion']=opt_portfolio.apply(lambda x: x.meth1_opt_weight*x.ASK_PRICE if x.meth1_opt_weight >0 else x.meth1_opt_weight*x.BID_PRICE,axis=1)
        opt_portfolio['meth2_bid_contribtion']=opt_portfolio.apply(lambda x: x.meth2_opt_weight*x.BID_PRICE if x.meth2_opt_weight >0 else x.meth2_opt_weight*x.ASK_PRICE,axis=1)
        opt_portfolio['meth2_ask_contribtion']=opt_portfolio.apply(lambda x: x.meth2_opt_weight*x.ASK_PRICE if x.meth2_opt_weight >0 else x.meth2_opt_weight*x.BID_PRICE,axis=1)
        opt_portfolio['meth3_bid_contribtion']=opt_portfolio.apply(lambda x: x.meth3_opt_weight*x.BID_PRICE if x.meth3_opt_weight >0 else x.meth3_opt_weight*x.ASK_PRICE,axis=1)
        opt_portfolio['meth3_ask_contribtion']=opt_portfolio.apply(lambda x: x.meth3_opt_weight*x.ASK_PRICE if x.meth3_opt_weight >0 else x.meth3_opt_weight*x.BID_PRICE,axis=1)
        S_method1_bid = opt_portfolio.meth1_bid_contribtion.sum()
        S_method1_ask = opt_portfolio.meth1_ask_contribtion.sum()
        S_method2_bid = opt_portfolio.meth2_bid_contribtion.sum()
        S_method2_ask = opt_portfolio.meth2_ask_contribtion.sum()
        S_method3_bid = opt_portfolio.meth3_bid_contribtion.sum()
        S_method3_ask = opt_portfolio.meth3_ask_contribtion.sum()
        return [S_method1_bid,S_method1_ask,
                S_method2_bid,S_method2_ask,
                S_method3_bid,S_method3_ask]

In [ ]:
df3[df3.date.isin(third_fridays)]
print(third_friday_holidays)
expir_not_third_friday=set(pd.to_datetime(df2.exdate.unique()).normalize())-set(third_fridays)
print(expir_not_third_friday)


## Whole Thing

In [6]:
#df3[df3.date.isin(third_fridays)].groupby(['date'])['exdate'].min()
#for fri in third_fridays

lb_year=2021
ub_year=2023
portfolio_dict={}
#portfolio_df=pd.DataFrame()
id_counter=1
for yr in range(lb_year,ub_year+1):
    print(f'{yr} file')
    df = option_metric_query('SPX',yr,day=None,month=None)
    df2=df.copy()
    lb='1996-01-04'
    ub='2040-12-31'
    #CBOE Holidays and Early Closes
    #   NOTE: need to come back to early closes
    cboe=mcal.get_calendar('CBOE_Index_Options')
    cboe_holidays=cboe.holidays()
    cboe_holidaylist=pd.to_datetime(cboe_holidays.holidays)
    fridays = list( pd.date_range(lb, ub,freq='W-FRI', tz='US/Eastern',normalize=True).values )
    third_fridays = list( pd.date_range(lb, ub,freq='WOM-3FRI', tz='US/Eastern',normalize=True).values )
    fridays=pd.to_datetime(fridays).normalize()
    third_fridays=pd.to_datetime(third_fridays).normalize()
    diffed_fridays=list(set(fridays)-set(third_fridays))
    #print(third_fridays)
    #print(cboe_holidaylist)
    holiday_dict={}
    third_friday_holidays=list(set(third_fridays) & set(cboe_holidaylist))
    #print(third_friday_holidays)
    expir_not_third_friday=set(pd.to_datetime(df2.exdate.unique()).normalize())-set(third_fridays)
    #print(expir_not_third_friday)
    #print('--------------------------')
    for expir in expir_not_third_friday:
        #print(expir in third_friday_holidays)
        for holiday in third_friday_holidays:
            if (holiday-expir).days==1:
                #print(holiday,expir)
                holiday_dict[expir]=True

    df2['holiday_exp']=df2['exdate'].map(holiday_dict)
    df2.fillna({'holiday_exp':False},inplace=True)

    df3=df2.copy()    
    df3['datetime_close']=df3['date']+pd.Timedelta(hours=16,minutes=15)
    df3['ex_time']=df3['exdate']+pd.Timedelta(hours=9,minutes=30)
    df3['time_to_exp']=df3['ex_time']-df3['datetime_close']

    df_fwd=get_fwd_price('SPX',yr)
    df4=df3.merge(df_fwd,how='left',left_on=['secid','date','exdate'],right_on=['secid','date','expiration'])
    #df4[pd.isnull(df4.forwardprice)]
    #df4['OOM']=df4.apply(lambda x:True if ((x.cp_flag=='C' and x.strike_price>=x.forwardprice) or (x.cp_flag=='P' and x.strike_price<=x.forwardprice)) else False )
    def oom_indicator(row):
        if (row.cp_flag=='C' and row.strike_price>=row.forwardprice) or (row.cp_flag=='P' and row.strike_price<=row.forwardprice):
            return True
        else:
            return False
    #df4['oom']=df4.apply(oom_indicator,axis=1)
    df4['midpoint_price']=(df4.best_bid+df4.best_offer)/2

    ##################


    for expir in sorted(df4.exdate.unique()):
        print(f'trade_date: {expir}')
        #display(df4[df4.date==expir].exdate.min())
        slice_=df4[df4.date==expir]
        next_expir = pd.Series(slice_.exdate.unique()).nsmallest(3)
        #portfolio_dict[expir]={}
        portfolio_dict[id_counter]={'Trade Date':expir}

        if not pd.isna(next_expir).all():
            #for term in next_expir:
            for i,term in enumerate(next_expir):
                #portfolio_dict[expir][f'{i+1} month expiration']=term
                portfolio_dict[id_counter][f'{i+1} month expiration']=term
                #print(term)
                term_df=slice_[(slice_.exdate==term)]
                #display(term_df)
                #print(3/0)
                ATM_strike_cands=term_df[~( (pd.isnull(term_df.best_bid)) | (pd.isnull(term_df.best_offer)) ) 
                            & ~(term_df.best_bid>term_df.best_offer) 
                            & ~(term_df.best_bid<=0)   ]
                
                mins_in_year=365*24*60
                mins_to_expir=ATM_strike_cands.time_to_exp.iloc[0].total_seconds()/60
                t=mins_to_expir/mins_in_year
                #print(ATM_strike_cands.iloc[0].time_to_exp.days)
                days_to_exp=ATM_strike_cands.iloc[0].time_to_exp.days
                #print(calculate_interest_rates(expir,ATM_strike_cands.iloc[0].time_to_exp.days))
                print(f'{ATM_strike_cands.iloc[0].time_to_exp.days} days to expiration')
                r=calculate_interest_rates(expir,ATM_strike_cands.iloc[0].time_to_exp.days)
                #missing data in yield curve fiel was throwing error
                #print(r)
                #print('~~~~~~~~~~~~~~~~~~')
                #display(ATM_strike_cands)

                def min_strike_diff(slice):
                    if ('C' in slice.cp_flag.unique()) and ('P' in slice.cp_flag.unique()):
                        return abs( slice[slice.cp_flag=='P'].midpoint_price.values[0] - slice[slice.cp_flag=='C'].midpoint_price.values[0])
                
                F_strike=ATM_strike_cands.groupby(['strike_price']).apply(min_strike_diff).idxmin()
                #print(F_strike)
                call_put_diff=ATM_strike_cands[(ATM_strike_cands.strike_price==F_strike)].sort_values(by='cp_flag')['midpoint_price'].diff().dropna().values[0]
                F=F_strike+np.exp(r*t)*call_put_diff
                K0=term_df[term_df.strike_price<=F].strike_price.max()
                #print(K0,F)
                #Which Fwd Price to Use
                #print('OptionMetrics Fwd:')
                #print(term_df.forwardprice.unique())
                #print('CBOE Fwd:')
                #print(F)

                def filter_included_options(opt_type):
                    if opt_type=='put':
                        OOM_opts=term_df[(term_df.strike_price<K0)&(term_df.cp_flag=='P')]
                        OOM_opts=OOM_opts.copy()
                        #OOM_opts.sort_values(by=['strike_price'],ascending=False,inplace=True) #sort upside down for puts
                        #print('puts')
                        #display(OOM_opts)
                        OOM_opts['excl_ind']=OOM_opts.best_bid.apply(lambda x: pd.isnull(x) or x<=0)
                        OOM_opts.sort_values(by=['strike_price'],ascending=False,inplace=True) #sort upside down for puts
                    else:
                        OOM_opts=term_df[(term_df.strike_price>K0)&(term_df.cp_flag=='C')]
                        OOM_opts=OOM_opts.copy()
                        #OOM_opts.sort_values(by=['strike_price'],ascending=True,inplace=True) #sort upside down for puts
                        #print('calls')
                        #display(OOM_opts)
                        OOM_opts['excl_ind']=OOM_opts.best_bid.apply(lambda x: pd.isnull(x) or x<=0)
                        #dont need to change sorting order for calls

                    OOM_opts['excl_ind']=OOM_opts['excl_ind'].cumsum()
                    #print(opt_type)
                    #display(OOM_opts[OOM_opts.excl_ind>=2])
                    incl_opts=OOM_opts[OOM_opts.excl_ind<2]
                    incl_opts=incl_opts[incl_opts.best_bid>0]
                    if opt_type=='put':
                        incl_opts.sort_values(by=['strike_price'],ascending=True,inplace=True)#change back
                    return incl_opts

                incl_puts,incl_calls=filter_included_options('put'),filter_included_options('call')
                #display(incl_puts)
                #display(incl_calls)
                
                df_K0=term_df[term_df.strike_price==K0]
                #display(df_K0[['optionid','best_bid','best_offer']])
                #print(4/0)
                pca=pd.DataFrame.from_dict({'strike_price':K0,
                                            'cp_flag':'P/C Avg', #put-call average
                                            'midpoint_price':term_df[(term_df.strike_price==K0)]['midpoint_price'].mean(),
                                            'best_bid':None,'best_offer':None,'optionid':None},
                                            orient='index').T

                opt_portfolio=pd.concat([incl_puts,pca,incl_calls])[['strike_price',
                                                                    'cp_flag',
                                                                    'midpoint_price',
                                                                    'best_bid','best_offer',
                                                                    'optionid']]
                #display(opt_portfolio)
                #display(opt_portfolio[opt_portfolio.strike_price==K0])
                opt_portfolio['dK']=(opt_portfolio.strike_price.shift(-1)-opt_portfolio.strike_price.shift(1))/2
                opt_portfolio=opt_portfolio.copy()
                #opt_portfolio.iloc[0]['dK']=opt_portfolio.iloc[1]['strike_price']-opt_portfolio.iloc[0]['strike_price']
                #opt_portfolio.iloc[-1]['dK']=opt_portfolio.iloc[-1]['strike_price']-opt_portfolio.iloc[-2]['strike_price']
                opt_portfolio.iloc[0,-1]=opt_portfolio.iloc[1]['strike_price']-opt_portfolio.iloc[0]['strike_price']
                opt_portfolio.iloc[-1,-1]=opt_portfolio.iloc[-1]['strike_price']-opt_portfolio.iloc[-2]['strike_price']
                #display(opt_portfolio)
                
                # VIX and SVIX
                #addign 2/t for now instead of 2
                VIX_opt_weight=(2/t)*opt_portfolio.dK/(opt_portfolio.strike_price**2)
                SVIX_opt_weight=(2/t)*opt_portfolio.dK/(F**2)
                # SKEW
                p1_option_weight=-opt_portfolio.dK/(opt_portfolio.strike_price**2)
                p2_option_weight=2*opt_portfolio.dK/(opt_portfolio.strike_price**2)
                #print( (opt_portfolio.strike_price/F).min() )
                #display(opt_portfolio[pd.isnull(opt_portfolio.strike_price)])
                #print(p2_option_weight)
                #print( 1-np.log(opt_portfolio.strike_price.astype(np.float64)/F) )
                p2_option_weight*=1-np.log(opt_portfolio.strike_price.astype(np.float64)/F)
                p3_option_weight=3*opt_portfolio.dK/(opt_portfolio.strike_price**2)
                p3_option_weight*=2*np.log(opt_portfolio.strike_price.astype(np.float64)/F)-np.log(opt_portfolio.strike_price.astype(np.float64)/F)**2

                e1=-(1+np.log(F/K0)-(F/K0))
                e2=2*np.log(K0/F)*(F/K0-1)+(1/2)*np.log(K0/F)**2
                e3=3*np.log(K0/F)**2 * (1/3*np.log(K0/F)-1+F/K0)
                P1=( np.exp(r*t)*(opt_portfolio.midpoint_price*p1_option_weight) ).sum()
                P1+=e1
                P2=( np.exp(r*t)*(opt_portfolio.midpoint_price*p2_option_weight) ).sum()
                P2+=e2
                P3=( np.exp(r*t)*(opt_portfolio.midpoint_price*p3_option_weight) ).sum()
                P3+=e3
                sigma=np.sqrt(P2-P1**2)
                
                opt_portfolio['VIX_opt_weight']=VIX_opt_weight*np.exp(r*t)
                opt_portfolio['SVIX_opt_weight']=SVIX_opt_weight*np.exp(r*t)
                opt_portfolio['meth1_opt_weight']=(p3_option_weight - 3*P1*p2_option_weight + 2*P1**2*p1_option_weight) * np.exp(r*t)/sigma**3
                opt_portfolio['meth2_opt_weight']=(-6*p1_option_weight - 3*p2_option_weight) * np.exp(r*t)/sigma**3
                opt_portfolio['meth3_opt_weight']=(SVIX_opt_weight-VIX_opt_weight) * 3/2*np.exp(r*t)/sigma**3
                #display(opt_portfolio)
                print( np.sqrt((opt_portfolio.VIX_opt_weight*opt_portfolio.midpoint_price).sum())*100 )
                print( np.sqrt((opt_portfolio.SVIX_opt_weight*opt_portfolio.midpoint_price).sum())*100 )
                #print( (opt_portfolio.VIX_opt_weight*opt_portfolio.midpoint_price).sum() )
                #print( (opt_portfolio.SVIX_opt_weight*opt_portfolio.midpoint_price).sum() )
                #print( 100-10*(opt_portfolio.meth1_opt_weight*opt_portfolio.midpoint_price).sum() )
                #print( 100-10*(opt_portfolio.meth2_opt_weight*opt_portfolio.midpoint_price).sum() )
                #print( 100-10*(opt_portfolio.meth3_opt_weight*opt_portfolio.midpoint_price).sum() )
                #print( (opt_portfolio.meth1_opt_weight*opt_portfolio.midpoint_price).sum() )
                #print( (opt_portfolio.meth2_opt_weight*opt_portfolio.midpoint_price).sum() )
                #print( (opt_portfolio.meth3_opt_weight*opt_portfolio.midpoint_price).sum() )
                
                #portfolio_dict[expir][f'{i+1} month VIX']=np.sqrt((opt_portfolio.VIX_opt_weight*opt_portfolio.midpoint_price).sum())*100
                #portfolio_dict[expir][f'{i+1} month SVIX']=np.sqrt((opt_portfolio.SVIX_opt_weight*opt_portfolio.midpoint_price).sum())*100
                portfolio_dict[id_counter][f'{i+1} month VIX']=np.sqrt((opt_portfolio.VIX_opt_weight*opt_portfolio.midpoint_price).sum())*100
                portfolio_dict[id_counter][f'{i+1} month SVIX']=np.sqrt((opt_portfolio.SVIX_opt_weight*opt_portfolio.midpoint_price).sum())*100

                """
                # at the end drop all the uneeded columsna and append the K0 cols at the end with 50/50 weight
                #print(opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].VIX_opt_weight)
                #print(opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].SVIX_opt_weight)
                #print(opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].dK)
                df_K0=df_K0[['strike_price','cp_flag','midpoint_price','best_bid','best_offer','optionid']]
                df_K0['dK']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].dK.values[0]
                #split the weight 50-50 for the ATM call and put
                df_K0['VIX_opt_weight']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].VIX_opt_weight.values[0]*1/2
                df_K0['SVIX_opt_weight']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].SVIX_opt_weight.values[0]*1/2
                df_K0['meth1_opt_weight']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].meth1_opt_weight.values[0]*1/2
                df_K0['meth2_opt_weight']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].meth2_opt_weight.values[0]*1/2
                df_K0['meth3_opt_weight']=opt_portfolio[opt_portfolio.cp_flag=='P/C Avg'].meth3_opt_weight.values[0]*1/2
                
                opt_portfolio=opt_portfolio[opt_portfolio.cp_flag!='P/C Avg']

                opt_portfolio_2=pd.concat([opt_portfolio,df_K0])[['strike_price',
                                                                    'cp_flag',
                                                                    'midpoint_price',
                                                                    'best_bid','best_offer',
                                                                    'optionid',
                                                                    'VIX_opt_weight',
                                                                    'SVIX_opt_weight',
                                                                    'meth1_opt_weight',
                                                                    'meth2_opt_weight',
                                                                    'meth3_opt_weight']]
                #display(opt_portfolio_2)
                opt_portfolio_2['portfolio_id']=id_counter
                #display(opt_portfolio_2.iloc[-1].VIX_opt_weight)
                opt_portfolio_2['VIX_trade_price']=opt_portfolio_2.apply(lambda x: x.best_offer if x.VIX_opt_weight >0 else x.best_bid,axis=1)
                opt_portfolio_2['SVIX_trade_price']=opt_portfolio_2.apply(lambda x: x.best_offer if x.SVIX_opt_weight >0 else x.best_bid,axis=1)
                opt_portfolio_2['meth1_trade_price']=opt_portfolio_2.apply(lambda x: x.best_offer if x.meth1_opt_weight >0 else x.best_bid,axis=1)
                opt_portfolio_2['meth2_trade_price']=opt_portfolio_2.apply(lambda x: x.best_offer if x.meth2_opt_weight >0 else x.best_bid,axis=1)
                opt_portfolio_2['meth3_trade_price']=opt_portfolio_2.apply(lambda x: x.best_offer if x.meth3_opt_weight >0 else x.best_bid,axis=1)

                opt_portfolio_2['trade_date']=expir
                opt_portfolio_2['portfolio_expiration']=closest_expir
                opt_portfolio_2['days_to_exp']=days_to_exp
                #this is cutting off the rounding
                #opt_portfolio_2.sort_values(by=['optionid'],inplace=True)
                #opt_portfolio_2=opt_portfolio_2.sort_values(by=['optionid'])
                

                print( (opt_portfolio_2.VIX_opt_weight*opt_portfolio_2.midpoint_price).sum() )
                print( (opt_portfolio_2.SVIX_opt_weight*opt_portfolio_2.midpoint_price).sum() )
                display(opt_portfolio_2)
                """
            id_counter+=1
                
        
        print('---------')

    print('~~~~~~~~~~~~~~~~~')

#portfolio_df
portfolio_dict

2021 file


/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_18654/2176133754.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2.fillna({'holiday_exp':False},inplace=True)


trade_date: 2021-01-15 00:00:00
34 days to expiration


/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_18654/2176133754.py:101: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  F_strike=ATM_strike_cands.groupby(['strike_price']).apply(min_strike_diff).idxmin()
/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_18654/2176133754.py:153: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  opt_portfolio=pd.concat([incl_puts,pca,incl_calls])[['strike_price',


23.635667122818894
21.928884699689732
62 days to expiration


...

17.061132377235243
16.303168203768433
62 days to expiration


/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_18654/2176133754.py:101: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  F_strike=ATM_strike_cands.groupby(['strike_price']).apply(min_strike_diff).idxmin()
/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_18654/2176133754.py:153: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  opt_portfolio=pd.concat([incl_puts,pca,incl_calls])[['strike_price',


18.42385286134479
17.003490424303983
90 days to expiration
19.22446976176927
17.355315972753928
---------
trade_date: 2023-09-15 00:00:00
---------
trade_date: 2023-10-20 00:00:00
---------
trade_date: 2023-11-17 00:00:00
---------
trade_date: 2023-12-15 00:00:00
---------
trade_date: 2024-01-19 00:00:00
---------
trade_date: 2024-02-16 00:00:00
---------
trade_date: 2024-03-15 00:00:00
---------
trade_date: 2024-04-19 00:00:00
---------
trade_date: 2024-05-17 00:00:00
---------
trade_date: 2024-06-21 00:00:00
---------
trade_date: 2024-07-19 00:00:00
---------
trade_date: 2024-08-16 00:00:00
---------
trade_date: 2024-09-20 00:00:00
---------
trade_date: 2024-12-20 00:00:00
---------
trade_date: 2025-06-20 00:00:00
---------
trade_date: 2025-12-19 00:00:00
---------
trade_date: 2026-12-18 00:00:00
---------
trade_date: 2027-12-17 00:00:00
---------
trade_date: 2028-12-15 00:00:00
---------
~~~~~~~~~~~~~~~~~


/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_18654/2176133754.py:101: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  F_strike=ATM_strike_cands.groupby(['strike_price']).apply(min_strike_diff).idxmin()
/var/folders/nr/239fpjgn5zg829rq2drl_2kw0000gn/T/ipykernel_18654/2176133754.py:153: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  opt_portfolio=pd.concat([incl_puts,pca,incl_calls])[['strike_price',


{1: {'Trade Date': Timestamp('2021-01-15 00:00:00'),
  '1 month expiration': Timestamp('2021-02-19 00:00:00'),
  '1 month VIX': np.float64(23.635667122818894),
  '1 month SVIX': np.float64(21.928884699689732),
  '2 month expiration': Timestamp('2021-03-19 00:00:00'),
  '2 month VIX': np.float64(25.784858577424362),
  '2 month SVIX': np.float64(23.150379200467047),
  '3 month expiration': Timestamp('2021-04-16 00:00:00'),
  '3 month VIX': np.float64(26.901556775129148),
  '3 month SVIX': np.float64(23.495280083597656)},
 2: {'Trade Date': Timestamp('2021-02-19 00:00:00'),
  '1 month expiration': Timestamp('2021-03-19 00:00:00'),
  '1 month VIX': np.float64(21.503559587567132),
  '1 month SVIX': np.float64(20.143817802740625),
  '2 month expiration': Timestamp('2021-04-16 00:00:00'),
  '2 month VIX': np.float64(24.533092979034915),
  '2 month SVIX': np.float64(21.993124476642286),
  '3 month expiration': Timestamp('2021-05-21 00:00:00'),
  '3 month VIX': np.float64(26.960690483526093),
 

In [10]:
#pd.DataFrame.from_dict(portfolio_dict).T.dropna(subset=['1 month expiration']).to_csv('vix_term_struct_sample.csv')
pd.DataFrame.from_dict(portfolio_dict).T.dropna(subset=['1 month expiration'])

,Trade Date,1 month expiration,1 month VIX,1 month SVIX,2 month expiration,2 month VIX,2 month SVIX,3 month expiration,3 month VIX,3 month SVIX
1,2021-01-15 00:00:00,2021-02-19 00:00:00,23.635667,21.928885,2021-03-19 00:00:00,25.784859,23.150379,2021-04-16 00:00:00,26.901557,23.49528
2,2021-02-19 00:00:00,2021-03-19 00:00:00,21.50356,20.143818,2021-04-16 00:00:00,24.533093,21.993124,2021-05-21 00:00:00,26.96069,23.274157
3,2021-03-19 00:00:00,2021-04-16 00:00:00,20.278635,19.062054,2021-05-21 00:00:00,24.252162,21.943821,2021-06-18 00:00:00,24.675604,21.737394
4,2021-04-16 00:00:00,2021-05-21 00:00:00,16.417661,15.594964,2021-06-18 00:00:00,18.799721,17.296342,2021-07-16 00:00:00,20.354231,18.230139
5,2021-05-21 00:00:00,2021-06-18 00:00:00,20.082735,18.822531,2021-07-16 00:00:00,21.890567,19.763416,2021-08-20 00:00:00,23.500381,20.700285
6,2021-06-18 00:00:00,2021-07-16 00:00:00,19.935515,18.416547,2021-08-20 00:00:00,21.433304,19.069091,2021-09-17 00:00:00,22.569209,19.729614
7,2021-07-16 00:00:00,2021-08-20 00:00:00,18.461915,17.099746,2021-09-17 00:00:00,20.611806,18.480787,2021-10-15 00:00:00,21.887509,19.106815
8,2021-08-20 00:00:00,2021-09-17 00:00:00,17.934365,16.636318,2021-10-15 00:00:00,20.266777,18.270098,2021-11-19 00:00:00,22.172289,19.361918
9,2021-09-17 00:00:00,2021-10-15 00:00:00,20.521596,18.917921,2021-11-19 00:00:00,22.084917,19.748901,2021-12-17 00:00:00,23.030351,20.170766
10,2021-10-15 00:00:00,2021-11-19 00:00:00,21.26376,20.385687,2021-12-17 00:00:00,19.654261,18.078426,2022-01-21 00:00:00,20.63364,18.284778


# Extra

In [11]:
df.exdate.unique()

<DatetimeArray>
['2022-01-21 00:00:00', '2022-02-18 00:00:00', '2022-03-18 00:00:00',
 '2022-04-14 00:00:00', '2022-05-20 00:00:00', '2022-06-17 00:00:00',
 '2022-07-15 00:00:00', '2022-08-19 00:00:00', '2022-09-16 00:00:00',
 '2022-10-21 00:00:00', '2022-11-18 00:00:00', '2022-12-16 00:00:00',
 '2023-01-20 00:00:00', '2023-03-17 00:00:00', '2023-06-16 00:00:00',
 '2023-12-15 00:00:00', '2024-12-20 00:00:00', '2025-12-19 00:00:00',
 '2026-12-18 00:00:00', '2023-02-17 00:00:00', '2023-04-21 00:00:00',
 '2023-05-19 00:00:00', '2023-09-15 00:00:00', '2023-07-21 00:00:00',
 '2024-06-21 00:00:00', '2027-12-17 00:00:00', '2023-08-18 00:00:00',
 '2023-10-20 00:00:00', '2024-01-19 00:00:00', '2023-11-17 00:00:00',
 '2024-03-15 00:00:00']
Length: 31, dtype: datetime64[ns]

In [15]:
#df#[df.am_settlement==1].exdate.unique()


lb=min(df.exdate)
ub=max(df.exdate)
#CBOE Holidays and Early Closes
#   NOTE: need to come back to early closes
cboe=mcal.get_calendar('CBOE_Index_Options')
cboe_holidays=cboe.holidays()
cboe_holidaylist=pd.to_datetime(cboe_holidays.holidays)
#sched=cboe.schedule(start_date=lb, end_date=ub)
#cboe.early_closes(schedule=sched)

#Times to Expiration
fridays = list( pd.date_range(lb, ub,freq='W-FRI', tz='US/Eastern',normalize=True).values )
third_fridays = list( pd.date_range(lb, ub,freq='WOM-3FRI', tz='US/Eastern',normalize=True).values )
fridays=pd.to_datetime(fridays).normalize()
third_fridays=pd.to_datetime(third_fridays).normalize()
diffed_fridays=list(set(fridays)-set(third_fridays))

print(third_fridays)
print('===============================')
display(sorted(df.exdate.unique()))

#df[df.am_settlement==1].exdate.unique()

DatetimeIndex(['2022-01-21', '2022-02-18', '2022-03-18', '2022-04-15',
               '2022-05-20', '2022-06-17', '2022-07-15', '2022-08-19',
               '2022-09-16', '2022-10-21', '2022-11-18', '2022-12-16',
               '2023-01-20', '2023-02-17', '2023-03-17', '2023-04-21',
               '2023-05-19', '2023-06-16', '2023-07-21', '2023-08-18',
               '2023-09-15', '2023-10-20', '2023-11-17', '2023-12-15',
               '2024-01-19', '2024-02-16', '2024-03-15', '2024-04-19',
               '2024-05-17', '2024-06-21', '2024-07-19', '2024-08-16',
               '2024-09-20', '2024-10-18', '2024-11-15', '2024-12-20',
               '2025-01-17', '2025-02-21', '2025-03-21', '2025-04-18',
               '2025-05-16', '2025-06-20', '2025-07-18', '2025-08-15',
               '2025-09-19', '2025-10-17', '2025-11-21', '2025-12-19',
               '2026-01-16', '2026-02-20', '2026-03-20', '2026-04-17',
               '2026-05-15', '2026-06-19', '2026-07-17', '2026-08-21',
      

[Timestamp('2022-01-21 00:00:00'),
 Timestamp('2022-02-18 00:00:00'),
 Timestamp('2022-03-18 00:00:00'),
 Timestamp('2022-04-14 00:00:00'),
 Timestamp('2022-05-20 00:00:00'),
 Timestamp('2022-06-17 00:00:00'),
 Timestamp('2022-07-15 00:00:00'),
 Timestamp('2022-08-19 00:00:00'),
 Timestamp('2022-09-16 00:00:00'),
 Timestamp('2022-10-21 00:00:00'),
 Timestamp('2022-11-18 00:00:00'),
 Timestamp('2022-12-16 00:00:00'),
 Timestamp('2023-01-20 00:00:00'),
 Timestamp('2023-02-17 00:00:00'),
 Timestamp('2023-03-17 00:00:00'),
 Timestamp('2023-04-21 00:00:00'),
 Timestamp('2023-05-19 00:00:00'),
 Timestamp('2023-06-16 00:00:00'),
 Timestamp('2023-07-21 00:00:00'),
 Timestamp('2023-08-18 00:00:00'),
 Timestamp('2023-09-15 00:00:00'),
 Timestamp('2023-10-20 00:00:00'),
 Timestamp('2023-11-17 00:00:00'),
 Timestamp('2023-12-15 00:00:00'),
 Timestamp('2024-01-19 00:00:00'),
 Timestamp('2024-03-15 00:00:00'),
 Timestamp('2024-06-21 00:00:00'),
 Timestamp('2024-12-20 00:00:00'),
 Timestamp('2025-12-

In [25]:
#set(third_fridays)-set(pd.to_datetime(df.exdate.unique()).normalize())
set(pd.to_datetime(df.exdate.unique()).normalize())-set(third_fridays)


#third_fridays

{Timestamp('2022-04-14 00:00:00')}